In [ ]:
import cv2
import mediapipe as mp
from ultralytics import YOLO
import numpy as np

### **Modelo**

In [ ]:
# Load the YOLOv8 model
model = YOLO('yolov8n.pt').to("cuda") # Load the model to GPU
mp_drawing = mp.solutions.drawing_utils

## **Detección de Face Mesh**

In [ ]:
# Initialize MediaPipe Pose
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1, min_detection_confidence=0.7)
face_connections = mp_face_mesh.FACEMESH_TESSELATION

In [ ]:
# Initialize video
cap = cv2.VideoCapture(0)

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    skeleton_mask = np.zeros_like(frame)

    # Run YOLO detection for person class
    results = model(frame, classes=[0])

    for result in results:
        for box in result.boxes:
            conf = float(box.conf[0])
            class_id = int(box.cls[0])
            if conf < 0.5 or class_id != 0:
                continue
            x_min, y_min, x_max, y_max = map(int, box.xyxy[0])

            # Box dimensions
            x_min = max(0, x_min - 10)
            y_min = max(0, y_min - 10)
            x_max = min(frame.shape[1], x_max + 10)
            y_max = min(frame.shape[0], y_max + 10)

            roi = frame[y_min:y_max, x_min:x_max]
            roi_rgb = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)

            face_results = face_mesh.process(roi_rgb)

            if face_results.multi_face_landmarks:
                for face_landmarks in face_results.multi_face_landmarks:
                    # Draw on main frame
                    mp_drawing.draw_landmarks(
                        image=frame[y_min:y_max, x_min:x_max],
                        landmark_list=face_landmarks,
                        connections=face_connections,
                        landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=1, circle_radius=1),
                        connection_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=1)
                    )

                    # Draw on skeleton mask
                    mp_drawing.draw_landmarks(
                        image=skeleton_mask[y_min:y_max, x_min:x_max],
                        landmark_list=face_landmarks,
                        connections=face_connections,
                        landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=1, circle_radius=1),
                        connection_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=1)
                    )

            # Draw bounding box
            cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)

    # Live view
    cv2.imshow('Face Mesh Detection', frame)
    cv2.imshow('Face Mask', skeleton_mask)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

## **Detección de Poses**

In [ ]:
# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False,
                    min_detection_confidence=0.7,
                    min_tracking_confidence=0.7)

pose_connections = mp_pose.POSE_CONNECTIONS

### **Inferencia en tiempo real**

In [ ]:
cap = cv2.VideoCapture(0)

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    skeleton_mask = np.zeros_like(frame)

    # Run YOLO detection
    results = model(frame, classes=[0])

    for result in results:
        for box in result.boxes:
            conf = float(box.conf[0])
            class_id = int(box.cls[0])
            if conf < 0.5 or class_id != 0:
                continue
            x_min, y_min, x_max, y_max = map(int, box.xyxy[0])

            # Expand box
            x_min = max(0, x_min - 10)
            y_min = max(0, y_min - 10)
            x_max = min(frame.shape[1], x_max + 10)
            y_max = min(frame.shape[0], y_max + 10)

            roi = frame[y_min:y_max, x_min:x_max]
            roi_rgb = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)

            pose_results = pose.process(roi_rgb)

            if pose_results.pose_landmarks:
                # Draw on main frame
                mp_drawing.draw_landmarks(
                    frame[y_min:y_max, x_min:x_max],
                    pose_results.pose_landmarks,
                    pose_connections,
                    landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=2),
                    connection_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2)
                )

                # Draw on skeleton mask
                mp_drawing.draw_landmarks(
                    skeleton_mask[y_min:y_max, x_min:x_max],
                    pose_results.pose_landmarks,
                    pose_connections,
                    landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=2),
                    connection_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2)
                )

            # Draw bounding box
            cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
            cv2.rectangle(skeleton_mask, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)

    # Live view
    cv2.imshow('Pose Estimation', frame)
    cv2.imshow('Skeleton Mask', skeleton_mask)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

### **Inferencia de video**

In [ ]:
# Input video
video_path = 'test.mp4'
cap = cv2.VideoCapture(video_path)

In [ ]:
# Get video info
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

In [ ]:
# Containers to store frames
annotated_frames = []
skeleton_masks = []

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    skeleton_mask = np.zeros_like(frame)

    # Run YOLO detection
    results = model(frame, classes=[0], augment=True)

    for result in results:
        for box in result.boxes:
            conf = float(box.conf[0])
            class_id = int(box.cls[0])
            if conf < 0.7 or class_id != 0:
                continue
            x_min, y_min, x_max, y_max = map(int, box.xyxy[0])

            # Initialize MediaPipe Pose
            x_min = max(0, x_min - 10)
            y_min = max(0, y_min - 10)
            x_max = min(frame.shape[1], x_max + 10)
            y_max = min(frame.shape[0], y_max + 10)

            roi = frame[y_min:y_max, x_min:x_max]
            roi_rgb = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)

            pose_results = pose.process(roi_rgb)

            if pose_results.pose_landmarks:
                # Draw on main frame
                mp_drawing.draw_landmarks(
                    frame[y_min:y_max, x_min:x_max],
                    pose_results.pose_landmarks,
                    pose_connections,
                    landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=2),
                    connection_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2)
                )

                # Draw on skeleton mask
                mp_drawing.draw_landmarks(
                    skeleton_mask[y_min:y_max, x_min:x_max],
                    pose_results.pose_landmarks,
                    pose_connections,
                    landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=2),
                    connection_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2)
                )

            # Draw bounding box
            cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
            cv2.rectangle(skeleton_mask, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)

    # Store both views
    annotated_frames.append(frame)
    skeleton_masks.append(skeleton_mask)

    # Optional live preview
    cv2.imshow('Pose Estimation', frame)
    cv2.imshow('Skeleton Mask', skeleton_mask)

cap.release()
cv2.destroyAllWindows()

# === Export videos ===

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out_pose = cv2.VideoWriter('output_pose2.mp4', fourcc, fps, (width, height))
out_mask = cv2.VideoWriter('output_mask2.mp4', fourcc, fps, (width, height))

for frame, mask in zip(annotated_frames, skeleton_masks):
    out_pose.write(frame)
    out_mask.write(mask)

out_pose.release()
out_mask.release()

print("Videos exported: output_pose.mp4 and output_mask.mp4")
